In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import math

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Activation
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.callbacks import ModelCheckpoint, EarlyStopping

%matplotlib inline

# Loading Dataset

In [3]:
stock_exchange_mk = pd.read_csv('data_mse_mse_historic_data_active.csv', sep='\t', parse_dates=['Date'])

In [4]:
df = pd.DataFrame(stock_exchange_mk)
df["stock_id"] = df["stock_id"].astype('category')
df.columns = df.columns.str.lower()

# Missing Dates

In [5]:
stocks = df.stock_id.unique()
dates_all = pd.period_range(min(df.date), max(df.date)).to_timestamp()
idx = pd.MultiIndex.from_product((dates_all, stocks), names=['date', 'stock_id'])
df_new = df.set_index(['date', 'stock_id']).reindex(idx).reset_index()

DROP UNNECESSARY DATES

In [6]:
df_list = []
for stock in df.stock_id.unique():
    each_df = df[df.stock_id == stock]
    groupby_stock = each_df.groupby([pd.PeriodIndex(data=each_df.date, freq='D'), 'stock_id'])
    df_list.append(groupby_stock)

In [7]:
stock_name = []
stock_start_date = []
for i in df_list:
    name = min(i.date)[0][1]
    stock_name.append(name)
    date = min(i.date)[0][0]
    stock_start_date.append(date)
    
start_date = dict(zip(stock_name, stock_start_date))

In [8]:
start_date_df = pd.DataFrame.from_dict(start_date, orient='index', dtype='datetime64[ns]', columns=['start_date'])

In [9]:
start_date_df.head()

,start_date
ALK,1997-01-09
BESK,2002-10-10
FERS,1999-04-15
GRNT,1999-10-28
KMB,1997-05-15


In [10]:
df_clean = df_new.set_index('stock_id').join(start_date_df, lsuffix='_filter', on='stock_id').reset_index()

In [11]:
df_clean.head()

,stock_id,date,open,high,low,close,volume,adj close,quantity,average,change %,volume total,ratio,ratio 1m,ratio 3m,ratio 6m,ratio 1y,ratio 2y,ratio 3y,start_date
0,ALK,1997-01-09,1070.0,1070.0,1070.0,1070.0,279270.0,1070.0,261.0,1070.0,NaN,279270.0,0.077,0.085,0.085,0.082,0.08,0.08,0.08,1997-01-09
1,BESK,1997-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-10-10
2,FERS,1997-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999-04-15
3,GRNT,1997-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999-10-28
4,KMB,1997-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-05-15


In [12]:
print(df_clean.shape, df_new.shape, df.shape)

(189860, 20) (189860, 19) (43441, 19)


In [13]:
df_drop = df_clean[df_clean['date'] >= df_clean['start_date']]

In [14]:
min(df_drop.date[df_drop.stock_id == 'BESK'])

Timestamp('2002-10-10 00:00:00')

FILLING DATA WITH ZERO VALUES

In [15]:
df_sorted = df_drop.sort_values(by=['stock_id', 'date'], ascending=True)
test = df_sorted.drop_duplicates()
print(df_sorted.shape, test.shape)

(164754, 20) (164754, 20)


In [16]:
df_sorted

,stock_id,date,open,high,low,close,volume,adj close,quantity,average,change %,volume total,ratio,ratio 1m,ratio 3m,ratio 6m,ratio 1y,ratio 2y,ratio 3y,start_date
0,ALK,1997-01-09,1070.0,1070.0,1070.0,1070.0,279270.0,1070.0,261.0,1070.00,NaN,279270.0,0.077,0.085,0.085,0.082,0.080,0.080,0.080,1997-01-09
22,ALK,1997-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
44,ALK,1997-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
66,ALK,1997-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
88,ALK,1997-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
110,ALK,1997-01-14,1070.0,1070.0,1050.0,1050.0,378370.0,1050.0,357.0,1059.86,1.0,378370.0,0.076,0.084,0.084,0.081,0.080,0.080,0.080,1997-01-09
132,ALK,1997-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
154,ALK,1997-01-16,1050.0,1030.0,1030.0,1030.0,246170.0,1030.0,239.0,1030.00,3.0,246170.0,0.074,0.082,0.082,0.079,0.077,0.077,0.077,1997-01-09
176,ALK,1997-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
198,ALK,1997-01-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09


In [17]:
df_sorted['close'] = df_sorted['close'].ffill()

In [18]:
df_sorted

,stock_id,date,open,high,low,close,volume,adj close,quantity,average,change %,volume total,ratio,ratio 1m,ratio 3m,ratio 6m,ratio 1y,ratio 2y,ratio 3y,start_date
0,ALK,1997-01-09,1070.0,1070.0,1070.0,1070.0,279270.0,1070.0,261.0,1070.00,NaN,279270.0,0.077,0.085,0.085,0.082,0.080,0.080,0.080,1997-01-09
22,ALK,1997-01-10,NaN,NaN,NaN,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
44,ALK,1997-01-11,NaN,NaN,NaN,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
66,ALK,1997-01-12,NaN,NaN,NaN,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
88,ALK,1997-01-13,NaN,NaN,NaN,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
110,ALK,1997-01-14,1070.0,1070.0,1050.0,1050.0,378370.0,1050.0,357.0,1059.86,1.0,378370.0,0.076,0.084,0.084,0.081,0.080,0.080,0.080,1997-01-09
132,ALK,1997-01-15,NaN,NaN,NaN,1050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
154,ALK,1997-01-16,1050.0,1030.0,1030.0,1030.0,246170.0,1030.0,239.0,1030.00,3.0,246170.0,0.074,0.082,0.082,0.079,0.077,0.077,0.077,1997-01-09
176,ALK,1997-01-17,NaN,NaN,NaN,1030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
198,ALK,1997-01-18,NaN,NaN,NaN,1030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09


In [21]:
from numpy import nan
df_sorted['low'] = np.where(df_sorted['low'].isnull().values.any(), df_sorted['close'], df_sorted['low'])

In [19]:
df_sorted['high'] = np.where(df_sorted['high'].isnull().values.any(), df_sorted['close'], df_sorted['high'])

In [22]:
df_sorted['open'] = np.where(df_sorted['open'].isnull().values.any(), df_sorted['close'], df_sorted['open'])

In [23]:
df_sorted

,stock_id,date,open,high,low,close,volume,adj close,quantity,average,change %,volume total,ratio,ratio 1m,ratio 3m,ratio 6m,ratio 1y,ratio 2y,ratio 3y,start_date
0,ALK,1997-01-09,1070.0,1070.0,1070.0,1070.0,279270.0,1070.0,261.0,1070.00,NaN,279270.0,0.077,0.085,0.085,0.082,0.080,0.080,0.080,1997-01-09
22,ALK,1997-01-10,1070.0,1070.0,1070.0,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
44,ALK,1997-01-11,1070.0,1070.0,1070.0,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
66,ALK,1997-01-12,1070.0,1070.0,1070.0,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
88,ALK,1997-01-13,1070.0,1070.0,1070.0,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
110,ALK,1997-01-14,1050.0,1050.0,1050.0,1050.0,378370.0,1050.0,357.0,1059.86,1.0,378370.0,0.076,0.084,0.084,0.081,0.080,0.080,0.080,1997-01-09
132,ALK,1997-01-15,1050.0,1050.0,1050.0,1050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
154,ALK,1997-01-16,1030.0,1030.0,1030.0,1030.0,246170.0,1030.0,239.0,1030.00,3.0,246170.0,0.074,0.082,0.082,0.079,0.077,0.077,0.077,1997-01-09
176,ALK,1997-01-17,1030.0,1030.0,1030.0,1030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09
198,ALK,1997-01-18,1030.0,1030.0,1030.0,1030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-01-09


# Feature Engineering

In [ ]:
# dates - (year, month, day etc...)
# rolling - (QTY, Volumne MKD etc...)
# target closing price - (1day, 7days, 30days, 2months etc)
# target QTY - (1day, 7days, 30days, 2months etc)